In [99]:
import pandas as pd
import numpy as np
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, MinMaxScaler, OneHotEncoder
from sklearn.feature_selection import SelectKBest, f_classif
from sklearn.decomposition import PCA
from sklearn.compose import ColumnTransformer
# from imblearn.over_sampling import SMOTE
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
import matplotlib.pyplot as plt
import seaborn as sns
# from yellowbrick import FeatureImportances
from yellowbrick.model_selection import LearningCurve
from yellowbrick.text import FreqDistVisualizer
# from feature_engine import missing_data_imputers as mdi
# from feature_engine import discretisers as dsc
# from feature_engine import encoding as ce
# from feature_engine import transformations as vt
import optuna


In [100]:
df = pd.read_csv('./LaqnData.csv')
df.head(),df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 105120 entries, 0 to 105119
Data columns (total 6 columns):
 #   Column                   Non-Null Count   Dtype  
---  ------                   --------------   -----  
 0   Site                     105120 non-null  object 
 1   Species                  105120 non-null  object 
 2   ReadingDateTime          105120 non-null  object 
 3   Value                    102564 non-null  float64
 4   Units                    105120 non-null  object 
 5   Provisional or Ratified  105120 non-null  object 
dtypes: float64(1), object(5)
memory usage: 4.8+ MB


(  Site Species   ReadingDateTime  Value   Units Provisional or Ratified
 0  EN5      NO  01/01/2022 00:00    0.7  ug m-3                       R
 1  EN5      NO  01/01/2022 00:15    0.5  ug m-3                       R
 2  EN5      NO  01/01/2022 00:30    0.3  ug m-3                       R
 3  EN5      NO  01/01/2022 00:45    NaN  ug m-3                       R
 4  EN5      NO  01/01/2022 01:00    NaN  ug m-3                       R,
 None)

Handling irrelavant column

In [101]:

for column in df.columns:
    if df[column].nunique(dropna=True)==1:
        df=df.drop(column,axis=1)
df

,Species,ReadingDateTime,Value,Units
0,NO,01/01/2022 00:00,0.7,ug m-3
1,NO,01/01/2022 00:15,0.5,ug m-3
2,NO,01/01/2022 00:30,0.3,ug m-3
3,NO,01/01/2022 00:45,NaN,ug m-3
4,NO,01/01/2022 01:00,NaN,ug m-3
...,...,...,...,...
105115,NOX,31/12/2022 22:45,12.2,ug m-3 as NO2
105116,NOX,31/12/2022 23:00,9.9,ug m-3 as NO2
105117,NOX,31/12/2022 23:15,9.6,ug m-3 as NO2
105118,NOX,31/12/2022 23:30,8.0,ug m-3 as NO2


Handling missing data


In [102]:


null_matrix =[]
null_threshold = 0.05

for column in df.columns:
    null_matrix.append(df[column].isnull().mean())
# print(null_matrix)
for i in range(0,len(null_matrix)):
    if np.issubdtype(df[df.columns[i]].dtype, np.character) or (null_matrix[i]<=null_threshold and null_matrix[i]!=0.0):
        df=df.dropna(subset=[df.columns[i]])
        df= df.reset_index(drop=True)
    elif null_matrix[i]>null_threshold:
        df[df.columns[i]] = df[df.columns[i]].fillna(df[df.columns[i]].mean())
    else:
        pass

df

,Species,ReadingDateTime,Value,Units
0,NO,01/01/2022 00:00,0.7,ug m-3
1,NO,01/01/2022 00:15,0.5,ug m-3
2,NO,01/01/2022 00:30,0.3,ug m-3
3,NO,01/01/2022 01:15,0.6,ug m-3
4,NO,01/01/2022 01:30,0.5,ug m-3
...,...,...,...,...
102559,NOX,31/12/2022 22:45,12.2,ug m-3 as NO2
102560,NOX,31/12/2022 23:00,9.9,ug m-3 as NO2
102561,NOX,31/12/2022 23:15,9.6,ug m-3 as NO2
102562,NOX,31/12/2022 23:30,8.0,ug m-3 as NO2
